In [1]:
# Import Dependencies
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Import the requests library.
import requests
# Import the API key.
from config import weather_api_key
# Import citipy
from citipy import citipy
# Import the datetime module from the datetime library.
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
coordinates = list(lat_lngs)

In [4]:
# Use the tuple() function to display the latitude and longitude combinations.
for coordinate in coordinates:
    print(coordinate[0], coordinate[1])

-84.54796489800354 15.520374352932691
-56.258021033724276 -92.33640752431641
67.13050391525616 8.663116806373779
77.24929659657494 -83.95766755411225
40.91883885420319 -90.54673827317754
49.56500914774648 100.03619858306439
-20.241448170196577 85.68800967110155
13.74190446714033 -141.45724030719592
-79.75003442457341 -82.52807160203244
-87.15791087981428 -75.43617077914153
-9.866154211432814 92.08947513216685
68.13339219984704 150.05146685897898
-72.40434830672486 -68.29443789210167
2.47649999094736 104.46816919358338
-67.76166724344648 -109.7063744006827
84.21105481947697 125.36088090340888
-58.462177076001446 -141.43102858139773
57.48766762204369 84.9252319359158
71.14616328798121 -92.48440239981643
78.99011311565334 -65.69858545133556
10.560244107448995 163.74253635667867
-37.05960270549481 47.82325806906934
-70.46831414316632 165.71765658231976
-89.36784123085624 -126.99815683932155
-52.05068603860157 13.305613506626884
84.96432902280941 -75.60693587779429
-16.401833474677574 31.13

1.78905101498556 107.99633756631528
87.31640553762543 -21.458362794237985
45.77251118457434 100.40315724585827
-42.27432980115294 154.3892478578261
-87.82994626567798 -61.2517468816916
77.5656732638925 102.47493690188344
34.61826644011896 4.220674896898402
85.15993130490179 -123.98949067610312
-86.22148877976143 57.43780026880509
86.92252229024524 15.062353657306431
10.881162795435117 -174.5802027252398
-37.307771228934016 95.21178002482083
26.9836067240622 -112.09854854763205
38.45642532989885 18.541994940111465
64.69266976800532 114.20338414473792
-41.5449036956378 75.7689710273971
71.2014204842632 81.3756742181265
15.43882005627195 13.096228517837432
79.2683729596557 55.33104865664737
30.539380867970763 149.69456053033218
54.05178397887124 -51.066617490817805
43.8441406278977 91.2583498100629
67.17709910954926 -22.66748219223217
45.00812050241345 175.56407746918137
31.32862802119223 178.55148590475955
65.67565628907357 -0.599388558625833
20.162265505148284 54.441589207471
22.3506856

-77.09962646930384 127.79698656126311
69.03504145108641 109.33971034442294
-73.32157251164242 29.10314231558604
-41.37471711354665 -2.3247321225905466
-58.505419122967794 20.78999593099786
84.39457296949135 70.60111330743752
72.15142223755811 -53.87120344395288
0.016147977725097462 46.48835238607592
49.4078693289666 -147.3960430998202
-40.51545879875796 -15.287830895548353
61.21291492785747 -34.49321118071478
68.98280318474036 55.119820055682936
27.130890461163403 148.45718873888262
27.248630359440554 16.944587583371487
-42.16022842038109 5.774864401508978
73.3910760928562 -170.1392660284242
-33.5989899457483 81.25615666436346
-80.50881077207589 9.700813167460609
19.80624440952785 56.25527394187645


In [5]:
# Starting URL for Weather Map API Call.
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

In [6]:
# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination.
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

630

In [8]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1

# Loop through all the cities in the list.
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes.
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    city_url = url + "&q=" + city.replace(" ","+")

    # Log the URL, record, and set numbers and the city.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count.
    record_count += 1
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")


Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | bredasdorp
Processing Record 2 of Set 1 | punta arenas
Processing Record 3 of Set 1 | sistranda
Processing Record 4 of Set 1 | qaanaaq
Processing Record 5 of Set 1 | monmouth
Processing Record 6 of Set 1 | moron
Processing Record 7 of Set 1 | hithadhoo
Processing Record 8 of Set 1 | hilo
Processing Record 9 of Set 1 | ushuaia
Processing Record 10 of Set 1 | bengkulu
Processing Record 11 of Set 1 | zyryanka
Processing Record 12 of Set 1 | mersing
Processing Record 13 of Set 1 | tiksi
Processing Record 14 of Set 1 | rikitea
Processing Record 15 of Set 1 | krasnyy yar
Processing Record 16 of Set 1 | thompson
Processing Record 17 of Set 1 | narsaq
Processing Record 18 of Set 1 | butaritari
Processing Record 19 of Set 1 | tsihombe
City not found. Skipping...
Processing Record 20 of Set 1 | bluff
Processing Record 21 of Set 1 | hermanus
Processing Record 22 of Set 1 | mount darwin
Processing Record 23 

Processing Record 38 of Set 4 | baykit
Processing Record 39 of Set 4 | suntar
Processing Record 40 of Set 4 | lagoa
Processing Record 41 of Set 4 | zoniana
Processing Record 42 of Set 4 | praia
Processing Record 43 of Set 4 | faya
Processing Record 44 of Set 4 | codrington
Processing Record 45 of Set 4 | yellowknife
Processing Record 46 of Set 4 | atar
Processing Record 47 of Set 4 | najran
Processing Record 48 of Set 4 | coihaique
Processing Record 49 of Set 4 | vestmannaeyjar
Processing Record 50 of Set 4 | ust-kuyga
Processing Record 1 of Set 5 | meulaboh
Processing Record 2 of Set 5 | gazojak
Processing Record 3 of Set 5 | lagos
Processing Record 4 of Set 5 | kodiak
Processing Record 5 of Set 5 | mount isa
Processing Record 6 of Set 5 | ulverstone
Processing Record 7 of Set 5 | andenes
Processing Record 8 of Set 5 | nizhneyansk
City not found. Skipping...
Processing Record 9 of Set 5 | erzin
Processing Record 10 of Set 5 | isangel
Processing Record 11 of Set 5 | mecca
Processing Re

Processing Record 32 of Set 8 | dicabisagan
Processing Record 33 of Set 8 | contamana
Processing Record 34 of Set 8 | chapais
Processing Record 35 of Set 8 | vysoke
Processing Record 36 of Set 8 | attendorn
Processing Record 37 of Set 8 | barranca
Processing Record 38 of Set 8 | awash
Processing Record 39 of Set 8 | kudahuvadhoo
Processing Record 40 of Set 8 | mogadishu
Processing Record 41 of Set 8 | grindavik
Processing Record 42 of Set 8 | seymchan
Processing Record 43 of Set 8 | college
Processing Record 44 of Set 8 | puerto del rosario
Processing Record 45 of Set 8 | emerald
Processing Record 46 of Set 8 | santa fe
Processing Record 47 of Set 8 | san miguel de cauri
Processing Record 48 of Set 8 | sunndalsora
Processing Record 49 of Set 8 | deputatskiy
Processing Record 50 of Set 8 | santa cruz de tenerife
Processing Record 1 of Set 9 | buritis
Processing Record 2 of Set 9 | grand river south east
City not found. Skipping...
Processing Record 3 of Set 9 | palabuhanratu
City not fo

Processing Record 18 of Set 12 | grandview
Processing Record 19 of Set 12 | mayor pablo lagerenza
Processing Record 20 of Set 12 | lamont
Processing Record 21 of Set 12 | nieuw amsterdam
Processing Record 22 of Set 12 | marsabit
Processing Record 23 of Set 12 | langsa
Processing Record 24 of Set 12 | san ramon
Processing Record 25 of Set 12 | winnemucca
Processing Record 26 of Set 12 | esso
Processing Record 27 of Set 12 | masvingo
Processing Record 28 of Set 12 | itaituba
Processing Record 29 of Set 12 | srednekolymsk
Processing Record 30 of Set 12 | severo-yeniseyskiy
Processing Record 31 of Set 12 | henties bay
Processing Record 32 of Set 12 | fort-shevchenko
Processing Record 33 of Set 12 | namtsy
Processing Record 34 of Set 12 | rungata
City not found. Skipping...
Processing Record 35 of Set 12 | puerto escondido
Processing Record 36 of Set 12 | kutum
Processing Record 37 of Set 12 | atasu
Processing Record 38 of Set 12 | turan
Processing Record 39 of Set 12 | harper
Processing Re

In [15]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bredasdorp,-34.5322,20.0403,50.29,95,36,2.59,ZA,2021-08-04 03:05:58
1,Punta Arenas,-53.1500,-70.9167,42.12,70,100,14.56,CL,2021-08-04 03:02:35
2,Sistranda,63.7256,8.8340,51.91,70,81,4.63,NO,2021-08-04 03:05:59
3,Qaanaaq,77.4840,-69.3632,42.40,95,100,1.79,GL,2021-08-04 03:05:59
4,Monmouth,40.3001,-74.1663,73.18,80,1,0.00,US,2021-08-04 03:05:59
5,Moron,-34.6534,-58.6198,53.98,95,31,2.82,AR,2021-08-04 03:05:59
6,Hithadhoo,-0.6000,73.0833,81.82,75,100,7.90,MV,2021-08-04 03:05:36
7,Hilo,19.7297,-155.0900,76.73,84,90,0.00,US,2021-08-04 03:03:29
8,Ushuaia,-54.8000,-68.3000,40.66,70,75,5.75,AR,2021-08-04 03:05:55
9,Bengkulu,-3.8004,102.2655,81.95,74,100,5.32,ID,2021-08-04 03:06:00


In [23]:
new_column_order = ["City","Country","Date","Lat","Lng","Max Temp","Humidity","Cloudiness","Wind Speed"]

In [25]:
city_data_df = city_data_df[new_column_order]
city_data_df.head(10)

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Bredasdorp,ZA,2021-08-04 03:05:58,-34.5322,20.0403,50.29,95,36,2.59
1,Punta Arenas,CL,2021-08-04 03:02:35,-53.1500,-70.9167,42.12,70,100,14.56
2,Sistranda,NO,2021-08-04 03:05:59,63.7256,8.8340,51.91,70,81,4.63
3,Qaanaaq,GL,2021-08-04 03:05:59,77.4840,-69.3632,42.40,95,100,1.79
4,Monmouth,US,2021-08-04 03:05:59,40.3001,-74.1663,73.18,80,1,0.00
5,Moron,AR,2021-08-04 03:05:59,-34.6534,-58.6198,53.98,95,31,2.82
6,Hithadhoo,MV,2021-08-04 03:05:36,-0.6000,73.0833,81.82,75,100,7.90
7,Hilo,US,2021-08-04 03:03:29,19.7297,-155.0900,76.73,84,90,0.00
8,Ushuaia,AR,2021-08-04 03:05:55,-54.8000,-68.3000,40.66,70,75,5.75
9,Bengkulu,ID,2021-08-04 03:06:00,-3.8004,102.2655,81.95,74,100,5.32


In [27]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
city_data_df.to_csv(output_data_file, index_label="City_ID")